<a href="https://colab.research.google.com/github/Javedz678/resume-screener/blob/main/ETL_Resume_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Extract text from resume</h1>

In [ ]:
!pip install sentence_transformers
!pip install pyresparser
!python -m spacy download en_core_web_sm
!pip install spacy
!pip install gingerit

In [29]:
import string
import re
import nltk
import statistics
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from tabulate import tabulate
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import os
from tqdm import tqdm
import en_core_web_sm
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
os.chdir("/content/resume/")
from pyresparser import ResumeParser
from gingerit.gingerit import GingerIt
import json

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
from tqdm.notebook import tqdm_notebook

In [31]:
#Parsing
def parse(path2file):
  return ResumeParser(path2file).get_extracted_data()

In [32]:
def start():
  resume_list = os.listdir()
  for resume in resume_list:
    print("Parsing.....")
    parse(resume)

In [33]:
#Global arrays
score_array=[]
master_records= []
top_institutes = ["IIT","NIT","IIIT","BITS"]
equvalent_courses = ["B.E","MSC","MCA","B.Tech","Bachelors","Information science","AI","ML"]
top_banks = ["JP Morgan","Goldman sachs","Axis bank","HSBC","VISA","PayPal","PWC","HDFC Bank"]
prefer = False

Data cleaning

In [34]:
def start(actual,expected):
  removeStopwords(actual,expected)

In [35]:
def checkTopInstitutes(education,requiredEducation):
  for institute in top_institutes:
    if institute in education:
      education = requiredEducation
      break
    else:
      education.join(',')

In [36]:
def checkEquvalentCourses(education,requiredEducation):
  for course in equvalent_courses:
    if course in education:
      education = requiredEducation
    else:
      education.join(',')

In [37]:
def preferExperience(candidateExperience):
  candidateExperience = candidateExperience.split()
  for bank in top_banks:
    if bank in candidateExperience:
      return True

In [38]:
def removeStopwords(actual,expected):
  stop_words = set(stopwords.words('english'))
  candidateExperienceTokens = word_tokenize(actual)
  requiredExperienceTokens = word_tokenize(expected)
  cleanCandidateExperience = [word for word in candidateExperienceTokens if not word in stopwords.words()]
  cleanrequiredExperience = [word for word in requiredExperienceTokens if not word in stopwords.words()]
  lemmatisation(cleanCandidateExperience,cleanrequiredExperience)

In [39]:
def lemmatisation(actual,expected):
  CandidateExperience = ' '.join(map(str, actual))
  RequiredExperience = ' '.join(map(str, expected))
  lemmatizer = WordNetLemmatizer()
  lemCandidateExperience =lemmatizer.lemmatize(CandidateExperience)
  lemRequiredExperience = lemmatizer.lemmatize(RequiredExperience)
  removePunctuation(lemCandidateExperience,lemRequiredExperience)

In [40]:
def removePunctuation(actual,expected):
  noPunCandidateExperience = ""
  noPunRequiredExperience = ""
  for character in actual:
      if character.isalnum():
          noPunCandidateExperience += character
      else:
        noPunCandidateExperience += " "
        
  for character in expected:
      if character.isalnum():
          noPunRequiredExperience += character
      else:
        noPunRequiredExperience += " "
  model(noPunCandidateExperience,noPunRequiredExperience)

In [41]:
def drawPieChart():
  y=[]
  for i in range(len(score_array)):
    if(i==2):
      y.append(score_array[i]*1.5)
    else:
      y.append(score_array[i])
  mylabels = ["Skills", "Education", "Experience"]
  explode = (0, 0, 0.1 ) 
  plt.pie(y, labels = mylabels, explode=explode, shadow=True)
  plt.title("Distribution of attributes of the Candidate.")
  plt.show() 

In [42]:
#Process prequesit requirements 
def processPrequesits(requireTopTierEducation,preferBankExperience):
  if(requireTopTierEducation):
    requiredEducation = "BE in computer science from" + ' '.join([str(elem) for elem in top_institutes])
    checkTopInstitutes(education,requiredEducation)
  else:
    requiredEducation = "BE in computer science "
    checkEquvalentCourses(education,requiredEducation)
  if(preferBankExperience):
    hasPreferance = preferExperience(candidateExperience)
    if(hasPreferance):
      prefer = True
  return requiredEducation

In [104]:
def initJob(requireTopTierEducation,preferBankExperience,requirementCorpus,prefer = False):
  #requiredEducation = processPrequesits(requireTopTierEducation,preferBankExperience)
  resume_list = os.listdir()
  for resume in tqdm_notebook(resume_list):
    obj = parse(resume)
    data = obj.get('experience')
    skills = obj.get('skills')
    allSkills = skills+data
    skillsCorpus = " ".join(allSkills)
    checkRequiredSkills(skillsCorpus)
    checkOptionalSkills(skillsCorpus)
    candidateCorpus = "".join(data)
    sentenceTranformerModel(candidateCorpus.lower(),requirementCorpus.lower())
    analytics(resume,prefer)
    print("---------------------------------------------")
  #drawPieChart()
  #analytics(prefer)

In [108]:
def checkRequiredSkills(data):
  rs = requiredSkills.split(",")
  countR= len(rs)
  counter=0
  data=data.lower()
  for skill in rs:
    if(skill.lower() in data):
        counter+=1

  if(counter>=countR):
    print("Has all the required skills")
  elif(counter>0 and counter<countR):
    print("Has partial required skills ")
  else:
    print("Does not have the required skills")
  

In [109]:
def checkOptionalSkills(data):
  rs= goodtohaveskills.split()
  countR= len(rs)
  counter=0
  data=data.lower()

  for skill in rs:
    if(skill.lower() in data):
        counter+=1

  if(counter>=countR):
    print("Has all the Optional skills")
  elif(counter>0 and counter<countR):
    print("Has partial Optional skills")
  else:
    print("Does not have the Optional skills")

In [46]:
def analytics(name,prefer = False):
  avg = score_array[0]
  score_array.append(name)
  if(avg>min_criteria):
    score_array.append("PASSED")
  elif(avg>70 and avg<min_criteria):
    score_array.append("Manual review needed")
  else:
    score_array.append("FAILED")

  if(prefer):
    score_array.append("Has banking experience.")
  print(score_array)
  score_array.clear()

In [47]:
def showTable():
  print(master_records)
  #print(tabulate(master_records, headers=["Resume score","Automatic screening","Insights"]))

<h3>Modelling</h3>

In [48]:
def sentenceTranformerModel(actual,expected):
  modelName = "bert-base-nli-mean-tokens"
  model = SentenceTransformer(modelName)
  vectorA = model.encode([actual])
  vectorB = model.encode([expected])
  similarity_index = cosine_similarity(vectorB, vectorA)
  final_score = similarity_index[0][0]*100
  score_array.append(int(final_score))

<h3>Analysis</h3>


Make changes below to describe what is needed.

## **Runnable**

***Organisation - Taken from front end***

In [49]:
requireTopTierEducation = False
preferBankExperience = False
min_criteria = 80
requiredEducation = "Bachelors of engineering "
requiredSkills = "Informatica Power Center 9.1.0/8.6.1,Shell scripting, SQL Server, Toad, Oracle 11g/10g/9i, Teradata, SQL, PL/SQL."
goodtohaveskills= "Unix, Main frame"
requiredExperience = "Experience: 5 – 9 years 1. Good understanding of Data Warehousing Concepts, Database design 2. Experience in ETL development and data migration experience. 3. Strong working experience on informatica power center tools 4. Experience on Teradata database, Tools & utilities 5. Strong experience in SQL and Stored procedures 6. Experience on Performance tuning on both Informatica and Teradata platform. 7. Good knowledge on shell scripting 8. Knowledge on GIT, any scheduling tool, Reporting environment 9. knowledge on the CICD process 10. Good to have exposure to Banking domain. 11. Understanding of Agile practices, Good knowledge on scrum and agile methodologies. 12. Experience and willingness to work in production environment."
requirementCorpus = requiredEducation + requiredSkills + requiredExperience

***Candidates - Taken from Resume***

In [111]:
#Start Job
initJob(requireTopTierEducation,preferBankExperience,requirementCorpus,prefer)

  0%|          | 0/6 [00:00<?, ?it/s]

Has partial skills 
Has partial Optional skills
[68, 'Yesuraju.pdf', 'FAILED']
---------------------------------------------
Has partial skills 
Has partial Optional skills
[72, 'RakeshM.pdf', 'Manual review needed']
---------------------------------------------
Has partial skills 
Does not have the Optional skills
[78, 'SivaKumar.pdf', 'Manual review needed']
---------------------------------------------
Has partial skills 
Has partial Optional skills
[78, 'Galla Renuka.pdf', 'Manual review needed']
---------------------------------------------
Has partial skills 
Has all the Optional skills
[62, 'VaraPrasad.pdf', 'FAILED']
---------------------------------------------
Has partial skills 
Has partial Optional skills
[75, 'Anamala SandeepReddy.pdf', 'Manual review needed']
---------------------------------------------
